<a href="https://colab.research.google.com/github/qcuongning/collab/blob/main/Thyroid_nodule_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Với các hàm cần sử dụng backend Keras mà bị lỗi K not contribute cần đổi về keras==2.2.4


The codebase is heavily inspired by the [respotory](https://github.com/qubvel/efficientnet)



# Get Data

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import json
import cv2
import csv
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications as app
seed = 1234
np.random.seed(seed)
from zipfile import ZipFile


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1bbKAqUuk7Y1q3xsDSwP07oOXN_GL3SQM'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('COVID-SemiSegz')
with ZipFile('COVID-SemiSegz', 'r') as tt:
  tt.extractall("")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#su dung khi dataset o trong drive neu khong bo qua
%cp drive/My\ Drive/COVID-SemiSeg.zip /content/
with ZipFile('COVID-SemiSeg.zip', 'r') as tt:
  tt.extractall("")

In [3]:
#su dung khi dataset o trong drive neu khong bo qua
%cp drive/My\ Drive/thyroid/thyroid.zip /content/
with ZipFile('thyroid.zip', 'r') as tt:
  tt.extractall("thyroid")

In [4]:
file_name = 'weights_save'
if not os.path.exists(file_name):
  os.mkdir(file_name)

# Initial Data

lấy các cặp ảnh và mask

ảnh resize về 192x288

mask là binary
đầu ra imgs , masks

In [94]:
def resize_pad(blob):
    high,width = blob.shape[:2]
    if high >= width:
        blob = cv2.resize(blob,(int(96*width/high),96))
        delta = 96 - 96*width/high
        left,right =  int(delta//2),int(delta//2)
        top,bottom  = 0, 0
    else:
        blob = cv2.resize(blob,(96,int(96*high/width)))
        delta = 96 - 96*high/width
        top,bottom = int(delta//2),int(delta//2)
        left,right = 0,0
    #imgs.append(blob)
    new_im = cv2.copyMakeBorder(blob, top, bottom, left, right, cv2.BORDER_CONSTANT,value=0)
    new_im = cv2.resize(new_im,(96,96))
    return new_im

In [ ]:
import xml.etree.ElementTree as ET

ff = os.listdir("thyroid")
folder = "thyroid/"
basics = []
context = []
margin = []
masks = []
labels = []
# h, w = 352 // 2, 512 // 2
for count, f in enumerate(ff):
    rd_num = np.random.uniform(0.7, 1)
    if f.find(".jpg") > 0:
        img = cv2.imread(folder + f)
        
        # img = cv2.resize(img, (w, h))
        ix_number = f.find("_")
        rep = f[ix_number + 1:ix_number + 2]
        if ix_number < 0:
            ix_number = f.find(".")
        number = f[:ix_number]
        if number == "127" or number == '54' or number == '165' or number == '176' or number == '203' or number == '205' or number == '166' or number == '197':
            continue
        root = ET.parse(folder + number + ".xml").getroot()
        lb = (root.find("tirads").text)
        if lb in ('4a','4b','4c','5'):
            tt = 1
        else:
            tt = 0
        for neighbor in root.findall("mark"):   # moi mark la mot anh trong mark co cac thuoc tinh </image>,</svg>
            if rep != neighbor.find("image").text:
                continue
            svg = neighbor.find("svg").text
            if svg is None:
                continue
            ss = eval(svg)
            for s in ss:
                if s["regionType"] =='arrow':
                    continue
                print(f)

                mask = np.zeros_like(img[:,:,0])
                contour = []
                for point in s["points"]:
                    x = point["x"]
                    y = point['y']
                    contour.append([[x, y]])
                contour = np.asarray(contour)
                cv2.drawContours(mask, [contour], 0, 255, 5)
                plt.imshow(mask)
                plt.show()
                rect = cv2.boundingRect(contour)
                x,y,w,h = rect
                if w<50 or h <50:
                    continue
                # if w >= h:   
                #     offset = w - h
                #     h1,h2,w1,w2 = y-offset//2-w//20,y+h+offset//2+w//20,x-w//20,x+w+w//20
                #     h1 = max(h1,0)
                #     w1 = max(w1,0)
                #     blob= img[h1:h2,w1:w2]
                # else:
                #     offset = h - w
                #     h1,h2,w1,w2 =y-h//20,y+h+h//20,x-offset//2-h//20,x+w+offset//2+h//20
                #     h1 = max(h1,0)
                #     w1 = max(w1,0)
                #     blob= img[h1:h2,w1:w2]
                h1,h2,w1,w2 = y-h//20,y+h+h//20,x-w//20,x+w+w//20
                blob= img[h1:h2,w1:w2]
                new_im = resize_pad(blob)
                basics.append(new_im)
                h1,h2,w1,w2 = max(y-h//5,0),y+h+h//5,max(x-w//5,0),x+w+w//5
                print(y-h//5,y+h+h//5,x-w//5,x+w+w//5)
                blob= img[h1:h2,w1:w2]
                new_im = resize_pad(blob)
                context.append(new_im)


                
                labels.append(tt)
basics = np.asarray(basics,dtype = np.float32)
labels = np.asarray(labels,dtype = np.float32)
#from sklearn.utils import shuffle
basics, labels = shuffle(basics, labels)

In [90]:
from keras.utils.np_utils import to_categorical  
labels = to_categorical(labels,2)
n = 550
x_train = imgs[:n]
x_dev = imgs[n:]

y_train = labels[:n]
y_dev = labels[n:]

In [46]:
from sklearn.metrics import f1_score, recall_score, precision_score

class show_metrics(tf.keras.callbacks.Callback):
    def __init__(self,x_dev,y_dev):
        super(show_metrics, self).__init__()
    def on_epoch_end(self, epoch, logs=None):
        y_score = self.model.predict(x_dev)
        print(f"f1 score:{f1_score(y_dev,y_score)}, recall: {recall_score(y_dev,y_score)}, precision: {precision_score(y_dev,y_score)}")


        

In [53]:
def f1(y_true,y_pred):
    return f1_score(y_true,y_pred)

In [9]:
from sklearn.model_selection import train_test_split
 #from itertools import izip
generator_x = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    #width_shift_range = 0.2,
    #height_shift_range = 0.2,
    shear_range = 0.1,
    zoom_range = 0.1,
    rotation_range = 180,
    fill_mode = 'reflect',
    #brightness_range = (0.9,1.1),
)
generator_y = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    #width_shift_range = 0.2,
    #height_shift_range = 0.2,
    shear_range = 0.1,
    zoom_range = 0.1,
    #brightness_range = (1.0,1.0),
    rotation_range = 180,
    fill_mode = 'reflect'
)
seed = 1234
batch = 16
#X_train, X_test, y_train, y_test = train_test_split(imgs, masks, test_size=0.1)
x_gen = generator_x.flow(x_train, batch_size = batch, shuffle = False, seed = seed)
#y_gen = generator_y.flow(y_train, batch_size = batch, shuffle = False, seed = seed)
train_generator = zip(x_gen, y_train)

# Fit model

Epoch 60/80
500/500 [==============================] - 288s 576ms/step - loss: 0.3507 - acc: 0.9603 - dice_coef: 0.9069 - jaccard_coef: 0.8409 - val_loss: 0.4036 - val_acc: 0.9290 - val_dice_coef: 0.8515 - val_jaccard_coef: 0.7673

In [92]:
input_ten=Input(shape=(96,96,3))
pretrain = app.ResNet50(include_top = False, weights='imagenet', input_tensor=input_ten)
pre_out=pretrain.get_layer("conv3_block4_out").output
flat=Flatten()(pre_out)
predict=Dense(1024,activation='relu')(flat)
predict = Dropout(0.5)(predict)
predict=Dense(512,activation='relu')(predict)
predict=Dense(256,activation='relu')(predict)
predict=Dense(2,activation='sigmoid')(predict)
model=Model(inputs=input_ten,outputs=predict)
sgd = RMSprop(lr = 0.0001)

model.compile(optimizer = sgd, loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

#lr_decay = lr_scheduler()
history = model.fit(x_train,y_train,batch_size=16, epochs = 50, 
                    validation_data = (x_dev, y_dev))

Epoch 1/50
35/35 [==============================] - 2s 52ms/step - loss: 3.5240 - accuracy: 0.5618 - val_loss: 1.2391 - val_accuracy: 0.5441
Epoch 2/50
35/35 [==============================] - 1s 36ms/step - loss: 1.5736 - accuracy: 0.5491 - val_loss: 0.9357 - val_accuracy: 0.5735
Epoch 3/50
35/35 [==============================] - 1s 36ms/step - loss: 1.2116 - accuracy: 0.6109 - val_loss: 0.9666 - val_accuracy: 0.5735
Epoch 4/50
35/35 [==============================] - 1s 36ms/step - loss: 1.0386 - accuracy: 0.6073 - val_loss: 1.5239 - val_accuracy: 0.4559
Epoch 5/50
35/35 [==============================] - 1s 37ms/step - loss: 0.8621 - accuracy: 0.6600 - val_loss: 1.3180 - val_accuracy: 0.5588
Epoch 6/50
35/35 [==============================] - 1s 36ms/step - loss: 0.7411 - accuracy: 0.7200 - val_loss: 0.9217 - val_accuracy: 0.5735
Epoch 7/50
35/35 [==============================] - 1s 36ms/step - loss: 0.5810 - accuracy: 0.7691 - val_loss: 0.8182 - val_accuracy: 0.5441
Epoch 8/50
35

Epoch 49/50
35/35 [==============================] - 1s 23ms/step - loss: 0.0108 - accuracy: 0.9964 - val_loss: 5.0408 - val_accuracy: 0.7358

pre_out=pretrain.get_layer("conv3_block4_out").output
flat=Flatten()(pre_out)
predict=Dense(256,activation='relu')(flat)
predict=Dense(2,activation='sigmoid')(predict)

In [69]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

y_pre=model.predict(x_dev)
y_pre=np.asarray(y_pre,dtype=np.float)

average_precision = average_precision_score(y_dev, y_pre)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.62
